# 나와 가까운 ```복권명당```은?

- 학과 : 컴퓨터공학과
- 학번 : 2017104025 
- 이름 : 정수연

# 1. 프로젝트 목표 및 내용
---

### 사용자가 입력한 주소를 통해 가장 가까우면서도 당첨이 많이 된 복권 명당을 추천해준다. 
### 기준 1) 사용자가 입력한 주소와의 거리
### 기준 2) 1등 당첨횟수

# 2. 주제 선정 이유
---
> **주제변경**  
> 이전에 제안한 ```메일분석 대시보드``` 프로젝트를 다른 수업에서도 진행하게 되어 중간보고서에서 아이디어를 변경했습니다.

### 복권을 구매하다보면 10번 구매했을때 하나도 당첨이 안되는 지점이 있는 반면 그래도 5등이라도 당첨되는 지점이 있었다. 여기서 "정말 복권명당이라는 곳이 있을까?" 혹은 "그럼 내 집에서 가까운 복권 명당은 어딜까?"라는 의문이 생겼다. 따라서 본 프로젝트에서 복권 1등을 가장 많이 배출한 지점을 알아볼 것이다. 또한 해당 리스트 내에서 사용자가 입력한 주소와 가장 가까운 복권 명당을 검색할 수 있도록 할 것이다.

# 3. 데이터 획득
---
### - 동행복권 : 현재까지 역대 당첨 지점 리스트에 대한 정보를 주소, 당첨횟수까지 얻을 수 있다. ([동행복권](https://dhlottery.co.kr/store.do?method=topStoreRank&rank=1&pageGubun=L645))

```프로젝트 진행 중 아래와 같은 문제가 있었고 동행복권 사이트에서도 오래된 회차까지의 정보를 얻을 수 있는 것을 알게되어 데이터 소스 페이지를 변경했다.```
#### 1) 사라진 상호 : 사라진 지점에 대한 정보도 있어 그에 대한 위도, 경도 업데이트가 불가했다.
#### 2) 데이터 중복 : 사라진 지점 문제 때문에 잘못된 정보가 쌓여 데이터 중복이 발생했다.


In [ ]:
# parser.py
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
import django

django.setup()
from crawler.models import LotteryList

KAKAO_API_KEY = os.environ.get("KAKAO_API_KEY", None)

def getLotteryList():
    driver = webdriver.Chrome('/Users/suyeon/Downloads/chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://dhlottery.co.kr/store.do?method=topStoreRank&rank=1&pageGubun=L645')
    Lottery = []
    for largePage in range(1, 22):
        if largePage == 1:
            first, last = 1, 11
        elif largePage == 21:
            first, last = 3, 4
        else:
            first, last = 3, 13
        print("first, last", first, last)
        for page in range(first, last):

            driver.find_element_by_xpath('//*[@id="page_box"]/a[{}]'.format(page)).send_keys(Keys.ENTER)
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            store_names = soup.select(
                '#article > div:nth-child(2) > div > div.group_content.group_data_search > table > tbody > tr > td:nth-child(2)'
            )

            locations = soup.select(
                '#article > div:nth-child(2) > div > div.group_content.group_data_search > table > tbody > tr > td:nth-child(4)'
            )

            counts = soup.select(
                '#article > div:nth-child(2) > div > div.group_content.group_data_search > table > tbody > tr > td:nth-child(3)'
            )

            for item in zip(store_names, locations, counts):
                lottery = {
                    'store_name': item[0].get_text().strip(),
                    'location': item[1].get_text().strip(),
                    'lottery_count': item[2].get_text().strip(),
                    'latitude': 0,
                    'longitude': 0
                }
                Lottery.append(lottery)
        if largePage == 21:
            break
        driver.find_element_by_css_selector('#page_box > a.go.next').send_keys(Keys.ENTER)
        driver.implicitly_wait(3)

    return Lottery

# 위도, 경도 업데이트
def getLatLng(input_list):
    for item in input_list:
        searchAddr = item['location']
        # location 으로 검색
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searchAddr)
        headers = {
            "Authorization": "KakaoAK {}".format(KAKAO_API_KEY)
        }
        # places : 검색결과
        places = requests.get(url, headers=headers).json()['documents']
        for place in places:
            item['latitude'] = place['x']
            item['longitude'] = place['y']
    return input_list


if __name__ == '__main__':
    lotteryList = getLotteryList()
    result = getLatLng(lotteryList)
    for item in lotteryList:
        LotteryList(store_name=item['store_name'], location=item['location'], lottery_count=item['lottery_count'],
                    latitude=item['latitude'], longitude=item['longitude']).save()


# 4. 구현 내용 설명 
---
### - 기술 스택
> frontend : html/css, Bootstrap  
> backend : python, Django

### - Django Model

```python
from django.db import models

class LotteryList(models.Model):
    store_name = models.CharField(max_length=50)
    location = models.CharField(max_length=100)
    lottery_count = models.IntegerField(default=0)
    latitude = models.FloatField(default=0)
    longitude = models.FloatField(default=0)
    def __str__(self):
        return self.store_name
    class Meta:
        db_table = 'lottery_store'
```

> 1. 상호명
> 2. 위치(도로명 주소)
> 3. 1등 당첨횟수
> 4. 위도
> 5. 경도

#### 크롤링을 통해 ```상호명, 위치, 1등 당첨횟수```에 대한 데이터를 얻었다.

### - 위도, 경도 업데이트

#### ```위도, 경도``` 데이터는 Kakao Local API 를 사용하여 LotteryList객체들에 업데이트한다. 
#### 앞서 받아온 데이터인 위치를 위도, 경도 검색시 활용 => ```주소 검색```
```python
def getLatLng(input_list):
    for item in input_list:
        searchAddr = item['location']
        # location 으로 검색
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searchAddr)
        headers = {
            "Authorization": "KakaoAK YOUR_API_KEY"
        }
        # places : 검색결과
        places = requests.get(url, headers=headers).json()['documents']
        for place in places:
            item['latitude'] = place['x']
            item['longitude'] = place['y']
    return input_list
```
#### 위도, 경도 정보는 후에 사용자로부터 입력받은 주소와의 거리 계산을 위해 활용된다.

### - 사용자가 입력한 주소와의 거리 계산
```python 
from haversine import haversine

def search(request):
    searchAddr = request.POST['user_address']
    # 사용자가 입력한 주소의 위도, 경도 받아오기

    lat, lng = getLatLng(searchAddr)
    userLocation = (float(lat), float(lng))

    # 위도, 경도 tuple 담긴 리스트
    locations = []


    # 기존 복권 판매소와의 거리 계산
    LotteryList.objects.all().order_by('id')

    for item in LotteryList.objects.all():
        locations.append((float(item.latitude), float(item.longitude)))

    distance = []

    for i in range(len(locations)):
        distance.append((i + 1, haversine(userLocation, locations[i])))

    distance.sort(key=lambda element:element[1])

    lottery_stores = []

    for i in range(len(distance)):
        lottery_stores.append(LotteryList.objects.get(id=distance[i][0]))

    context = {
        "lottery_stores": lottery_stores
    }
    # distance 기준으로 정렬한 리스트 html로 넘기기

    return render(request, 'search.html', context=context)
```


### - 메인 기능

#### 1. 복권명당 랭킹
<img src="main.png" width="650px" height="500px" alt="main page"></img><br/>

#### 2. 검색어와 가장 가까운 복권명당 랭킹
1) '광진구 자양번영로 28-4'로 검색한 결과  
<img src="search1.png" width="650px" height="500px" alt="main page"></img><br/>

2) '경기도 용인시 기흥구 덕영대로 1732'로 검색한 결과  
<img src="search2.png" width="650px" height="500px" alt="main page"></img><br/>




# 5. 사용 방법
---

```bash
$ conda create --name [YOUR_ENV_NAME] python=3.6
$ conda activate [YOUR_ENV_NAME]
$ pip install -r requirements.txt
$ python parser.py
$ python manage.py makemigrations
$ python manage.py migrate
$ python manage.py runserver
```

